# 线性回归的简介实现

随着深度学习框架的发展，开发深度学习应用变得越来越便利。实践中，我们通常可以用比上一节更简介的代码来实现同样的模型。在本节中，我们将介绍如何使用MXNet提供Gluon接口更方便地实现线性回归的训练

## 生成数据集

我们生成与上一节中相同的数据集。其中features是训练数据特征，labels是标签

In [1]:
from mxnet import autograd, nd

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = nd.random.normal(scale=1, shape = (num_examples, num_inputs))
labels = true_w[0] * features[:,0] + true_w[1] * features[:,1]+true_b
labels += nd.random.normal(scale= 0.01, shape = labels.shape)

## 读取数据

Gluon提供了data包来读取数据。 由于data常用作变量名，我们将导入的data模块用添加了Gluon首字母的假名gdata代替。在每一次迭代中，我们将随机读取包含10个数据样本的小批量

In [2]:
from mxnet.gluon import data as gdata

batch_size = 10

#将训练数据的特征和标签组合
dataset = gdata.ArrayDataset(features, labels)
#  随机读取小批量
data_iter = gdata.DataLoader(dataset, batch_size, shuffle = True)

这里data_iter的使用跟上一节中的一样。 让我们读取并打印第一个小批量数据样本

In [3]:
for X, y in data_iter:
    print(X, y)
    break


[[-0.46044326 -0.8689304 ]
 [-0.10685237  0.3870012 ]
 [ 1.0069948  -1.3342417 ]
 [ 1.2852769  -1.8211212 ]
 [ 0.4326006   0.9435222 ]
 [ 0.3889665  -0.6177039 ]
 [-0.08295607 -0.5464172 ]
 [-1.2302126  -1.1133662 ]
 [-0.38817322 -0.35244492]
 [ 0.7099881   0.03483092]]
<NDArray 10x2 @cpu(0)> 
[ 6.229307   2.6694508 10.749971  12.984802   1.8577776  7.069023
  5.87861    5.531663   4.608702   5.4816027]
<NDArray 10 @cpu(0)>


## 定义模型

在上一节从零开始的实现中，我们需要定义模型参数，并使用它们一步步描述是怎么计算的。当模型结构变得更复杂时，这些步骤将变得更繁琐。其实，Gluon提供了大量预定义的层，这使我们只需关注使用哪些层来构造模型。下面将介绍如何使用Gluon更简介地定义线性回归

首先，导入nn模块，实际上，nn是netural networks的缩写，顾名思义，该模块定义了大量神经网络的层，我们先定义一个模型变量net它是一个Sequential 实例，在Gluon中，Sequential实例可以看做是一个串联各个层的容器，在构造模型时，我们在该容器中一次添加层，当给定输入数据时，容器中的每一层将一次计算并将输出作为下一层的输入。


In [4]:
from mxnet.gluon import nn

net = nn.Sequential()

回顾图3.1中线性回归 在神将网络图中的表示，作为一个单层神经网络，线性回归输出层中的神经元和输入层中各个输入完全连接，因此，线性回归的输出层又叫全连接层，在Gluon中，全连接层是一个Dense实例，我们定义该层输出个数为1

In [5]:
net.add(nn.Dense(1))

值得一提的是， 在Gluon中我们无须制定每一层输入的形状，例如线性回归的输入个数，当模型得到数据时，例如后面执行net(X)时， 模型将自动推断出每一层的输入个数，我们将在之后“深度学习计算”一章详细介绍这种机制   Gluon的质疑设计为模型开发带来了便利

## 初始化模型参数

在使用net前，我们需要初始化模型参数，如线性回归模型中的权重和偏差，我们从MXNet导入init模块。该模块提供了模型参数初始化的各种方法，这里的init事initializer的缩写。我们通过init.Normal(sigma = 0.01) 指定权重参数每个元素将在初始化时随机采样与均值为0 标准差为0.01的正态分布。偏差参数默认会初始化为零

In [6]:
from mxnet import init

net.initialize(init.Normal(sigma = 0.01))

## 定义损失函数

在Gluon中，loss模块定义了各种损失函数。我们用假名gloss代替导入的 loss模块，并直接使用它提供的平方损失作为模型的损失函数

In [7]:
from mxnet.gluon import loss as gloss

loss = gloss.L2Loss()  # 平方损失又称L2范数损失

## 定义优化算法

同样，我们也无须实现小批量随机梯度下降，在导入Gluon后，我们创建一个Trainer实例，并制定学习率为0.03的小批量随机梯度下降sgd为优化算法。该优化算法将用来迭代net实例所有通过add函数嵌套的层所包含的全部参数，这些参数可以通过collect_params函数获取

In [8]:
from mxnet import gluon

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

## 训练模型

在使用Gluon训练模型时，我们通过调用Trainer实例的step函数来迭代模型参数。上一节中我们提到，由于变量l是长度为batch_size的一维NDArray，执行l.backward() 等价于执行l.sum().backward().按照小批量随机梯度下降的定义，我们在step函数中指明批量大小，从而对批量中样本梯度求平均

In [9]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

epoch 1, loss: 0.035129
epoch 2, loss: 0.000129
epoch 3, loss: 0.000048


下面我们分别比较学到的模型参数和真实的模型参数， 我们从net获得需要的层，并访问其权重(weight)和偏差(bias)。学到的参数和真实的参数很接近

In [10]:
dense = net[0]
true_w, dense.weight.data()

([2, -3.4], 
 [[ 1.9998782 -3.399732 ]]
 <NDArray 1x2 @cpu(0)>)

In [11]:
true_b, dense.bias.data()


(4.2, 
 [4.2001157]
 <NDArray 1 @cpu(0)>)